In [1]:
import pandas as pd
import geopandas as gpd

# step 0 : process the basic point data with geometry type
## Basic data: location data on the street in Seattle (PNT)

In [2]:
# import csv data
pnt =pd.read_csv('./data/csv/point_data.csv')

In [4]:
# convert csv data to geodataframe, using geometry columns in the original .csv file
pnt_gdf = gpd.GeoDataFrame(pnt, geometry=gpd.points_from_xy(pnt.lon, pnt.lat),crs="EPSG:4326")
pnt_gdf

,id,panoid,lat,lon,year,geometry
0,0,NIaPIeovdkNBHVN5iVnztA,47.680257,-122.275383,2019,POINT (-122.27538 47.68026)
1,1,18eJo43CcDp-OYgv6WQ6_A,47.679680,-122.275363,2019,POINT (-122.27536 47.67968)
2,2,mPS-qH3BPKxmZSDx5pDl4g,47.680108,-122.275400,2019,POINT (-122.27540 47.68011)
3,3,RDxIDku0QLZJQuen7EU71Q,47.680404,-122.276173,2019,POINT (-122.27617 47.68040)
4,4,1vod9wfz0Zzziz5lteVUyQ,47.680403,-122.275642,2019,POINT (-122.27564 47.68040)
...,...,...,...,...,...,...
63818,63818,PXZEu_slfRwDshdmOb8fZQ,47.680174,-122.316830,2019,POINT (-122.31683 47.68017)
63819,63819,EP-X34MVVYXTmO73onKUjw,47.680188,-122.317209,2019,POINT (-122.31721 47.68019)
63820,63820,tt4Xq7zVoPonVsY3KJxRCQ,47.559415,-122.315488,2019,POINT (-122.31549 47.55941)
63821,63821,Jr0OC55RGZmkU-CyIS3Fwg,47.559424,-122.315885,2019,POINT (-122.31589 47.55942)


## Basic data: Seattle Census Block Group (POLY, 481 in total).

In [6]:
cbg = gpd.read_file('./data/shp/sea_cbg_4326.shp')
#inspect the columns and keep the interested one
lis = list(cbg.columns)
print (lis)

['OBJECTID', 'TRACTCE10', 'BLKGRPCE10', 'GEOID10', 'NAMELSAD10', 'INTPTLAT10', 'INTPTLON10', 'TRACT', 'TRBG', 'TRBG_STR', 'ACRES_TOTA', 'ACRES_LAND', 'ACRES_WATE', 'WATER', 'SHAPE_Leng', 'SHAPE_Area', 'OBJECTID_1', 'GEOID10_1', 'TRFIPS', 'CFIPS', 'SFIPS', 'CSA', 'CSA_Name', 'CBSA', 'CBSA_Name', 'COUNTHU10', 'TOTPOP10', 'HH', 'WORKERS', 'AC_TOT', 'AC_WATER', 'AC_LAND', 'AC_UNPR', 'D2A_EPHHM', 'D2B_E8MIXA', 'D3b', 'D4a', 'D2A_Ranked', 'D2B_Ranked', 'D4A_Ranked', 'D3B_Ranked', 'WalkIndex', 'Shape_Le_1', 'Shape_Ar_1', 'geometry']


In [7]:
cbg = cbg[['GEOID10', 'TRACTCE10', 'BLKGRPCE10', 'NAMELSAD10', 'INTPTLAT10', 'INTPTLON10', 'TRACT', 'TRBG', 'TRBG_STR', 'ACRES_TOTA', 'ACRES_LAND', 'ACRES_WATE', 'WATER', 'geometry']]
# check the sample data, or also you can check the coordinate system of this geodataframe by using cbg_gdf.crs
cbg.head(5)

,GEOID10,TRACTCE10,BLKGRPCE10,NAMELSAD10,INTPTLAT10,INTPTLON10,TRACT,TRBG,TRBG_STR,ACRES_TOTA,ACRES_LAND,ACRES_WATE,WATER,geometry
0,530330001001,000100,1,Block Group 1,+47.7276870,-122.2815160,100,100.1,100.1,588.283002,221.041200,367.285577,0,"POLYGON ((-122.26840 47.72641, -122.26719 47.7..."
1,530330001002,000100,2,Block Group 2,+47.7293165,-122.2924690,100,100.2,100.2,71.258030,71.263215,0.000000,0,"POLYGON ((-122.28970 47.73193, -122.28965 47.7..."
2,530330001003,000100,3,Block Group 3,+47.7228093,-122.2894558,100,100.3,100.3,91.604629,91.611355,0.000000,0,"POLYGON ((-122.28633 47.72093, -122.28629 47.7..."
3,530330001004,000100,4,Block Group 4,+47.7319367,-122.2944531,100,100.4,100.4,31.237245,31.239309,0.000000,0,"POLYGON ((-122.29237 47.73194, -122.29231 47.7..."
4,530330001005,000100,5,Block Group 5,+47.7229283,-122.2944374,100,100.5,100.5,56.956546,56.960756,0.000000,0,"POLYGON ((-122.29273 47.72019, -122.29273 47.7..."


## Basic data: 10-m buffer zone from point data

In [53]:
# create Buffer zone GeoSeries
pnt_gdf_gs_10 = pnt_gdf.buffer(0.0001)  #'0.0001' degree ~= 10m

# create Buffer zone Geodataframe by setting "geometry=GeoSeries"
buf_gdf_10 = gpd.GeoDataFrame(geometry=pnt_gdf_gs_10) 

# merge 2 Geodataframe : Basic point Geodataframe (based) and Buffer Geodataframe by using "on=..."
pnt_gdf_10_new = pnt_gdf.merge(buf_gdf_10, on=buf_gdf_10.index)

# since the type of what is done after merge is "Dataframe", we need to set "geometry='geometry_y'"
pnt_gdf_buf_10 = gpd.GeoDataFrame(data=pnt_gdf_10_new, geometry='geometry_y')
pnt_gdf_buf_10.head(5)

,key_0,id,panoid,lat,lon,year,geometry_x,dis_ospace_m,geometry_y
0,0,0,NIaPIeovdkNBHVN5iVnztA,47.680257,-122.275383,2019,POINT (-122.27538 47.68026),330.953852,"POLYGON ((-122.27528 47.68026, -122.27528 47.6..."
1,1,1,18eJo43CcDp-OYgv6WQ6_A,47.679680,-122.275363,2019,POINT (-122.27536 47.67968),331.191554,"POLYGON ((-122.27526 47.67968, -122.27526 47.6..."
2,2,2,mPS-qH3BPKxmZSDx5pDl4g,47.680108,-122.275400,2019,POINT (-122.27540 47.68011),329.327785,"POLYGON ((-122.27530 47.68011, -122.27530 47.6..."
3,3,3,RDxIDku0QLZJQuen7EU71Q,47.680404,-122.276173,2019,POINT (-122.27617 47.68040),271.962764,"POLYGON ((-122.27607 47.68040, -122.27607 47.6..."
4,4,4,1vod9wfz0Zzziz5lteVUyQ,47.680403,-122.275642,2019,POINT (-122.27564 47.68040),311.804808,"POLYGON ((-122.27554 47.68040, -122.27554 47.6..."


## Basic data: 50-m buffer zone from point data

In [14]:
pnt_gdf_gs_50 = pnt_gdf.buffer(0.0005)  #'0.0005' degree ~= 50m
buf_gdf_50 = gpd.GeoDataFrame(geometry=pnt_gdf_gs_50) 
img_gdf_50_new = pnt_gdf.merge(buf_gdf_50, on=buf_gdf_50.index)
img_gdf_buf_50 = gpd.GeoDataFrame(data=img_gdf_50_new, geometry='geometry_y')
img_gdf_buf_50.head(5)

,key_0,id,panoid,lat,lon,year,geometry_x,geometry_y
0,0,0,NIaPIeovdkNBHVN5iVnztA,47.680257,-122.275383,2019,POINT (-122.27538 47.68026),"POLYGON ((-122.27488 47.68026, -122.27489 47.6..."
1,1,1,18eJo43CcDp-OYgv6WQ6_A,47.679680,-122.275363,2019,POINT (-122.27536 47.67968),"POLYGON ((-122.27486 47.67968, -122.27487 47.6..."
2,2,2,mPS-qH3BPKxmZSDx5pDl4g,47.680108,-122.275400,2019,POINT (-122.27540 47.68011),"POLYGON ((-122.27490 47.68011, -122.27490 47.6..."
3,3,3,RDxIDku0QLZJQuen7EU71Q,47.680404,-122.276173,2019,POINT (-122.27617 47.68040),"POLYGON ((-122.27567 47.68040, -122.27568 47.6..."
4,4,4,1vod9wfz0Zzziz5lteVUyQ,47.680403,-122.275642,2019,POINT (-122.27564 47.68040),"POLYGON ((-122.27514 47.68040, -122.27514 47.6..."


# Step 1. Find the nearest distance from each point to ooen space

In [15]:
from shapely.ops import nearest_points
from geopy.distance import lonlat, distance

In [16]:
# define a function to calculate the nearest distance from point (Dataframe format) to the polygon (Dataframe format) 
# return type will be also in Dataframe format
def nearest(df_target,df_poi,column_name='dis'): # return df_target GeoDataFrame with a new column of nearest distance in meter
    dis_lis=[] # use list to store aa column of value: 'dis_openSpace' in m(meters)
    poi_union = df_poi.unary_union #<class 'shapely.geometry.multipolygon.MultiPolygon'>
    for geom in df_target['geometry']:    
        near = nearest_points(geom, poi_union) #return 2 points making up the nearest distance
        two_Pts=[]
        for pt in near:
            target= (pt.x,pt.y) #use tuple to store x,y coordinate of class objects
            two_Pts.append(target)
        dis = distance(lonlat(*two_Pts[0]), lonlat(*two_Pts[1])).m #https://geopy.readthedocs.io/en/stable/#module-geopy.distance
        dis_lis.append(dis)
    df_target[column_name]=dis_lis
    return df_target

In [18]:
# Read open space file (polygon Geodataframe)
ospace = gpd.read_file('./data/shp/sea_ospace.shp') # since the Seattle City only offers the park boundaries, so the open space means parks here.
ospace.head(5)

,OBJECTID,NAME,PMA,PARKSBND_A,SHAPE_Leng,SHAPE_Area,geometry
0,1,12TH AVE SQUARE PARK,4467,7316.172948,342.909025,7316.143683,"POLYGON ((-122.31625 47.60714, -122.31638 47.6..."
1,2,14TH AVENUE NW BOAT RAMP,4010,27888.351315,758.646958,27888.239762,"POLYGON ((-122.37380 47.66139, -122.37339 47.6..."
2,3,17TH AVENUE NE CENTERSTRIP,4160,71950.878767,5902.227161,71950.590966,"MULTIPOLYGON (((-122.30952 47.66651, -122.3095..."
3,4,3001 E MADISON,296,14679.930267,578.567011,14679.871548,"POLYGON ((-122.29280 47.62507, -122.29276 47.6..."
4,5,48TH AVE SW/SW CHARLESTOWN ST,4590,14467.911693,481.835246,14467.853821,"POLYGON ((-122.39356 47.56986, -122.39407 47.5..."


In [21]:
# calculate the nearest function using nearest(df,df,self-defined-name)
dis_result = nearest(pnt_gdf,ospace,'dis_ospace_m')

In [22]:
dis_result.head(5)

,id,panoid,lat,lon,year,geometry,dis_ospace_m
0,0,NIaPIeovdkNBHVN5iVnztA,47.680257,-122.275383,2019,POINT (-122.27538 47.68026),330.953852
1,1,18eJo43CcDp-OYgv6WQ6_A,47.679680,-122.275363,2019,POINT (-122.27536 47.67968),331.191554
2,2,mPS-qH3BPKxmZSDx5pDl4g,47.680108,-122.275400,2019,POINT (-122.27540 47.68011),329.327785
3,3,RDxIDku0QLZJQuen7EU71Q,47.680404,-122.276173,2019,POINT (-122.27617 47.68040),271.962764
4,4,1vod9wfz0Zzziz5lteVUyQ,47.680403,-122.275642,2019,POINT (-122.27564 47.68040),311.804808


In [24]:
# spatial join 2 Geodataframe : Basic cbg Geodataframe (based) and dis_result Geodataframe by using "how='left'"
#by using "how='left'", we keep ALL the based cbg Geodataframe plus newly added columns from dis_result
sjoin_ospace = gpd.sjoin(left_df=cbg, right_df=dis_result,how='left')

# groupby and calculating mean to determine the value of the column "dis_ospace_m" for each row in cbg_gdf
df_dis_ospace_m = sjoin_ospace.groupby(['GEOID10'])['dis_ospace_m'].mean().reset_index()
df_dis_ospace_m

,GEOID10,dis_ospace_m
0,530330001001,194.450279
1,530330001002,292.763196
2,530330001003,221.587445
3,530330001004,140.243242
4,530330001005,270.273622
...,...,...
476,530330121002,265.544826
477,530330260011,238.622362
478,530330264004,NaN
479,530330264005,NaN


In [62]:
#Add the new column to the original data
cbg_with_dis_result = cbg.merge(df_dis_ospace_m, on='GEOID10')
cbg_with_dis_result.head(5)

,GEOID10,TRACTCE10,BLKGRPCE10,NAMELSAD10,INTPTLAT10,INTPTLON10,TRACT,TRBG,TRBG_STR,ACRES_TOTA,ACRES_LAND,ACRES_WATE,WATER,geometry,dis_ospace_m
0,530330001001,000100,1,Block Group 1,+47.7276870,-122.2815160,100,100.1,100.1,588.283002,221.041200,367.285577,0,"POLYGON ((-122.26840 47.72641, -122.26719 47.7...",194.450279
1,530330001002,000100,2,Block Group 2,+47.7293165,-122.2924690,100,100.2,100.2,71.258030,71.263215,0.000000,0,"POLYGON ((-122.28970 47.73193, -122.28965 47.7...",292.763196
2,530330001003,000100,3,Block Group 3,+47.7228093,-122.2894558,100,100.3,100.3,91.604629,91.611355,0.000000,0,"POLYGON ((-122.28633 47.72093, -122.28629 47.7...",221.587445
3,530330001004,000100,4,Block Group 4,+47.7319367,-122.2944531,100,100.4,100.4,31.237245,31.239309,0.000000,0,"POLYGON ((-122.29237 47.73194, -122.29231 47.7...",140.243242
4,530330001005,000100,5,Block Group 5,+47.7229283,-122.2944374,100,100.5,100.5,56.956546,56.960756,0.000000,0,"POLYGON ((-122.29273 47.72019, -122.29273 47.7...",270.273622


# Step 2. Find the speed limit of the street which each point is located on

In [64]:
# speed as source data
speed_limi = gpd.read_file("./data/shp/sea_streets_4326.shp")
speed = speed_limi[['SPEEDLIMIT','geometry']]
speed #feature data (for it's attribute only, not geomtry(POLYLINE)

,SPEEDLIMIT,geometry
0,25,"LINESTRING (-122.33754 47.60612, -122.33820 47..."
1,25,"LINESTRING (-122.34001 47.60882, -122.34092 47..."
2,20,"LINESTRING (-122.35538 47.62637, -122.35538 47..."
3,20,"LINESTRING (-122.35558 47.63912, -122.35557 47..."
4,20,"LINESTRING (-122.35553 47.64562, -122.35553 47..."
...,...,...
23801,20,"LINESTRING (-122.28081 47.54244, -122.27975 47..."
23802,20,"LINESTRING (-122.35958 47.71781, -122.36093 47..."
23803,35,"LINESTRING (-122.36207 47.70173, -122.36086 47..."
23804,20,"LINESTRING (-122.29413 47.70113, -122.29342 47..."


In [65]:
# spatial join: which street segment(s) are within each 10-m buffer zone for each point? op can be any of this: contains, intersects, within
sjoin_speed = gpd.sjoin(left_df=pnt_gdf_buf_10, right_df=speed,op='intersects') 
#group by, using 'mean' *index_right is from the index of speed limit polyline feature, so use 'key_0' beacuae it is the index of the original image point feature
s_speedlimit = sjoin_speed.groupby(['key_0'])['SPEEDLIMIT'].mean() #Series, 36569 intotal
# s_speedlimit.value_counts() #for each value, how many numbers are there
# transform Series to Datafame
df_speedlimit = s_speedlimit.to_frame() 
df_speedlimit #Dataframe

C:\Users\skfh\Anaconda3\envs\GEOG573\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


,SPEEDLIMIT
key_0,
0,20.0
1,20.0
2,20.0
5,25.0
6,25.0
...,...
63818,20.0
63819,20.0
63820,20.0


In [69]:
#merge 2 dataframe using the column names 
pnt_df_speedlimit = pnt.merge(df_speedlimit, left_on='id', right_on='key_0')

#because the merge result is dataframe, we need to transform Dataframe to Geodataframe
pnt_gdf_speedlimit = gpd.GeoDataFrame(pnt_df_speedlimit,geometry='geometry')
pnt_gdf_speedlimit.head(5)

,id,panoid,lat,lon,year,geometry,dis_ospace_m,SPEEDLIMIT
0,0,NIaPIeovdkNBHVN5iVnztA,47.680257,-122.275383,2019,POINT (-122.27538 47.68026),330.953852,20.0
1,1,18eJo43CcDp-OYgv6WQ6_A,47.679680,-122.275363,2019,POINT (-122.27536 47.67968),331.191554,20.0
2,2,mPS-qH3BPKxmZSDx5pDl4g,47.680108,-122.275400,2019,POINT (-122.27540 47.68011),329.327785,20.0
3,5,ti0JBP3Gvr6yMtM_2KF46A,47.533600,-122.336093,2019,POINT (-122.33609 47.53360),304.551551,25.0
4,6,iW1roUgFVX4ghSxGyfT-Ig,47.532488,-122.335328,2019,POINT (-122.33533 47.53249),230.514092,25.0


In [71]:
# spatial join 2 Geodataframe : Basic cbg Geodataframe (based) and image_gdf_speedlimit dataframe by using "how='left'"
sjoin_slimit = gpd.sjoin(left_df=cbg, right_df=pnt_gdf_speedlimit,how='left')
df_speedlimit = sjoin_slimit.groupby(['GEOID10'])['SPEEDLIMIT'].mean().reset_index()
df_speedlimit

C:\Users\skfh\Anaconda3\envs\GEOG573\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(epsg:4326 != None)
  "(%s != %s)" % (left_df.crs, right_df.crs)


,GEOID10,SPEEDLIMIT
0,530330001001,20.176056
1,530330001002,23.868421
2,530330001003,23.880597
3,530330001004,25.107527
4,530330001005,24.865591
...,...,...
476,530330121002,21.419048
477,530330260011,20.000000
478,530330264004,NaN
479,530330264005,NaN


# Step 3. Crime using buffer(50m) from each location (PNT)

In [29]:
crim_all = pd.read_csv("./data/csv/sea_crim.csv")
crim_all

,Report Number,Offense ID,Report DateTime,Crime Against Category,Offense Parent Group,Offense Code,MCPP,100 Block Address,Longitude,Latitude
0,2019-000004,7667965226,1/1/2019 0:03,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,290,CAPITOL HILL,10XX BLOCK OF E PINE ST,-122.318813,47.615250
1,2019-000008,7657304625,1/1/2019 0:17,PERSON,ASSAULT OFFENSES,13B,CAPITOL HILL,15XX BLOCK OF BROADWAY,-122.320790,47.614655
2,2019-000018,7650534021,1/1/2019 2:15,PERSON,ASSAULT OFFENSES,13B,ALKI,11XX BLOCK OF ALKI AVE SW,-122.386797,47.595148
3,2019-000020,7696850850,1/1/2019 0:15,PERSON,ASSAULT OFFENSES,13B,QUEEN ANNE,4XX BLOCK OF BROAD ST,-122.348134,47.620292
4,2019-000023,7685549985,1/1/2019 0:15,PROPERTY,ROBBERY,120,JUDKINS PARK/NORTH BEACON HILL,20TH AVE S / S MAIN ST,-122.306355,47.599997
...,...,...,...,...,...,...,...,...,...,...
121334,2020-922255,15685450912,9/25/2020 14:23,PROPERTY,LARCENY-THEFT,23G,NORTHGATE,8XX BLOCK OF NE 90TH ST,-122.318959,47.694020
121335,2020-922256,15685451780,9/25/2020 14:23,PROPERTY,LARCENY-THEFT,23H,UNIVERSITY,MONTLAKE BLVD NE / NE PACIFIC PL,-122.303907,47.651083
121336,2020-922312,15694875238,9/26/2020 14:22,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,290,FIRST HILL,BOREN AVE / BROADWAY,-122.320745,47.604531
121337,2020-968854,15595904149,9/17/2020 5:02,PROPERTY,LARCENY-THEFT,23C,NORTHGATE,132XX BLOCK OF AURORA AVE N,-122.344997,47.725036


In [32]:
#only collect for 2019 report
crim_all['Year'] = crim_all['Report Number'].str[:4] #Extract first 4 characters
crim = crim_all.loc[crim_all['Year'] == '2019'] 
crim_gdf = gpd.GeoDataFrame(crim, geometry=gpd.points_from_xy(crim.Longitude, crim.Latitude),crs="EPSG:4326")
crim_gdf.head(5)

,Report Number,Offense ID,Report DateTime,Crime Against Category,Offense Parent Group,Offense Code,MCPP,100 Block Address,Longitude,Latitude,Year,geometry
0,2019-000004,7667965226,1/1/2019 0:03,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,290,CAPITOL HILL,10XX BLOCK OF E PINE ST,-122.318813,47.615250,2019,POINT (-122.31881 47.61525)
1,2019-000008,7657304625,1/1/2019 0:17,PERSON,ASSAULT OFFENSES,13B,CAPITOL HILL,15XX BLOCK OF BROADWAY,-122.320790,47.614655,2019,POINT (-122.32079 47.61466)
2,2019-000018,7650534021,1/1/2019 2:15,PERSON,ASSAULT OFFENSES,13B,ALKI,11XX BLOCK OF ALKI AVE SW,-122.386797,47.595148,2019,POINT (-122.38680 47.59515)
3,2019-000020,7696850850,1/1/2019 0:15,PERSON,ASSAULT OFFENSES,13B,QUEEN ANNE,4XX BLOCK OF BROAD ST,-122.348134,47.620292,2019,POINT (-122.34813 47.62029)
4,2019-000023,7685549985,1/1/2019 0:15,PROPERTY,ROBBERY,120,JUDKINS PARK/NORTH BEACON HILL,20TH AVE S / S MAIN ST,-122.306355,47.599997,2019,POINT (-122.30636 47.60000)


In [33]:
crim_gdf = crim_gdf[['Offense ID','geometry']]
#spatial join for Crime: How many crime incident are there around within 50-m buffer zone area for each location?
sjoin_crim_50 = gpd.sjoin(left_df=img_gdf_buf_50, right_df=crim_gdf) 
sjoin_crim_50 #geodataframe already

C:\Users\skfh\Anaconda3\envs\GEOG573\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != EPSG:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


,key_0,id,panoid,lat,lon,year,geometry_x,geometry_y,index_right,Offense ID
0,0,0,NIaPIeovdkNBHVN5iVnztA,47.680257,-122.275383,2019,POINT (-122.27538 47.68026),"POLYGON ((-122.27488 47.68026, -122.27489 47.6...",16659,8195480487
2,2,2,mPS-qH3BPKxmZSDx5pDl4g,47.680108,-122.275400,2019,POINT (-122.27540 47.68011),"POLYGON ((-122.27490 47.68011, -122.27490 47.6...",16659,8195480487
3,3,3,RDxIDku0QLZJQuen7EU71Q,47.680404,-122.276173,2019,POINT (-122.27617 47.68040),"POLYGON ((-122.27567 47.68040, -122.27568 47.6...",16659,8195480487
4,4,4,1vod9wfz0Zzziz5lteVUyQ,47.680403,-122.275642,2019,POINT (-122.27564 47.68040),"POLYGON ((-122.27514 47.68040, -122.27514 47.6...",16659,8195480487
6,6,6,iW1roUgFVX4ghSxGyfT-Ig,47.532488,-122.335328,2019,POINT (-122.33533 47.53249),"POLYGON ((-122.33483 47.53249, -122.33483 47.5...",67475,12119923741
...,...,...,...,...,...,...,...,...,...,...
63804,63804,63804,BzBQwuwLZc--MzmRY4FAvQ,47.682614,-122.326207,2019,POINT (-122.32621 47.68261),"POLYGON ((-122.32571 47.68261, -122.32571 47.6...",65640,11860155560
63814,63814,63814,tq7mbbR_nbW8fp66z8O58Q,47.680182,-122.317900,2017,POINT (-122.31790 47.68018),"POLYGON ((-122.31740 47.68018, -122.31740 47.6...",65633,11859978657
63815,63815,63815,pzvFlRSVf9h8j0KK5kfEtg,47.680184,-122.318292,2019,POINT (-122.31829 47.68018),"POLYGON ((-122.31779 47.68018, -122.31779 47.6...",65633,11859978657
63817,63817,63817,KdVWUTGGzjRCWC9JoSRy3A,47.680169,-122.316297,2019,POINT (-122.31630 47.68017),"POLYGON ((-122.31580 47.68017, -122.31580 47.6...",53869,7689587795


In [34]:
#group by, using 'mean' use 'key_0' beacuae it is the index of the original image point feature
s_crim_50 = sjoin_crim_50.groupby(['key_0'])['index_right'].count() #Series, 36569 intotal
df_crim_50 = s_crim_50.to_frame() #Series to df
df_crim_50 = df_crim_50.rename(columns={"index_right": "crim_count_50m"}) #change column name
df_crim_50

,crim_count_50m
key_0,
0,1
2,1
3,1
4,1
6,2
...,...
63816,3
63817,1
63818,1


In [46]:
# merge 2 dataframe using the column names 
pnt_df_crim_50 = pnt.merge(df_crim_50, left_on='id', right_on='key_0')
# transform Dataframe to Geodataframe
pnt_gdf_crim_50 = gpd.GeoDataFrame(pnt_df_crim_50,geometry='geometry')

print ('type of pnt_df_crim_50:', type(pnt_df_crim_50))
print ('type of pnt_gdf_crim_50:', type(pnt_gdf_crim_50))

type of pnt_df_crim_50: <class 'pandas.core.frame.DataFrame'>
type of pnt_gdf_crim_50: <class 'geopandas.geodataframe.GeoDataFrame'>


In [48]:
# spatial join 2 Geodataframe : Basic cbg Geodataframe (based) and pnt_gdf_crim_50 Geodataframe by using "how='left'"
sjoin_crim_50 = gpd.sjoin(left_df=cbg, right_df=pnt_gdf_crim_50,how='left')
df_crim_50 = sjoin_crim_50.groupby(['GEOID10'])['crim_count_50m'].mean()
df_crim_50 #Series

C:\Users\skfh\Anaconda3\envs\GEOG573\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(epsg:4326 != None)
  "(%s != %s)" % (left_df.crs, right_df.crs)


GEOID10
530330001001     1.830189
530330001002     5.296296
530330001003    11.650000
530330001004     8.285714
530330001005    21.243902
                  ...    
530330121002     1.620690
530330260011          NaN
530330264004          NaN
530330264005          NaN
530330265001    21.000000
Name: crim_count_50m, Length: 481, dtype: float64

# Step 4. Count of Trees using buffer(10m) from each location (PNT)

In [51]:
tree_gdf = gpd.read_file('./data/shp/sea_tree.shp')
tree_gdf.head(5)

,OBJECTID,geometry
0,1,POINT (-122.34418 47.61150)
1,2,POINT (-122.33642 47.61334)
2,3,POINT (-122.35039 47.68176)
3,5,POINT (-122.37046 47.68811)
4,6,POINT (-122.31331 47.61766)


In [54]:
#spatial join: how many trees (PNT) are within the points' buffer zone area? 
sjoin_tree = gpd.sjoin(left_df=pnt_gdf_buf_10, right_df=tree_gdf)
sjoin_tree #gpd already

C:\Users\skfh\Anaconda3\envs\GEOG573\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


,key_0,id,panoid,lat,lon,year,geometry_x,dis_ospace_m,geometry_y,index_right,OBJECTID
14,14,14,do3fTG9zjX82icqmAFHBqg,47.573209,-122.339254,2019,POINT (-122.33925 47.57321),1317.900270,"POLYGON ((-122.33915 47.57321, -122.33915 47.5...",130604,168911
14,14,14,do3fTG9zjX82icqmAFHBqg,47.573209,-122.339254,2019,POINT (-122.33925 47.57321),1317.900270,"POLYGON ((-122.33915 47.57321, -122.33915 47.5...",130605,168912
32,32,32,xY8UvRZxdzY802eC-cJ8Ug,47.607564,-122.289455,2019,POINT (-122.28946 47.60756),241.937250,"POLYGON ((-122.28936 47.60756, -122.28936 47.6...",8640,11324
33,33,33,#NAME?,47.607036,-122.289458,2019,POINT (-122.28946 47.60704),191.415931,"POLYGON ((-122.28936 47.60704, -122.28936 47.6...",10266,13466
33,33,33,#NAME?,47.607036,-122.289458,2019,POINT (-122.28946 47.60704),191.415931,"POLYGON ((-122.28936 47.60704, -122.28936 47.6...",75248,101257
...,...,...,...,...,...,...,...,...,...,...,...
63822,63822,63822,V-2kOfPa83TB02jXysTKEA,47.518702,-122.274263,2016,POINT (-122.27426 47.51870),212.799718,"POLYGON ((-122.27416 47.51870, -122.27416 47.5...",126114,163640
63822,63822,63822,V-2kOfPa83TB02jXysTKEA,47.518702,-122.274263,2016,POINT (-122.27426 47.51870),212.799718,"POLYGON ((-122.27416 47.51870, -122.27416 47.5...",126115,163641
63822,63822,63822,V-2kOfPa83TB02jXysTKEA,47.518702,-122.274263,2016,POINT (-122.27426 47.51870),212.799718,"POLYGON ((-122.27416 47.51870, -122.27416 47.5...",126116,163642
63822,63822,63822,V-2kOfPa83TB02jXysTKEA,47.518702,-122.274263,2016,POINT (-122.27426 47.51870),212.799718,"POLYGON ((-122.27416 47.51870, -122.27416 47.5...",126117,163643


In [55]:
s_tree_10= sjoin_tree.groupby(['key_0'])['OBJECTID'].count()
df_tree_10 = s_tree_10.to_frame() #Series to df
df_tree_10 = df_tree_10.rename(columns={"OBJECTID": "tree_count_10m"}) #change column name
df_tree_10.head(5)

,tree_count_10m
key_0,
14,2
32,1
33,3
37,1
38,3


In [58]:
#merge 2 dataframe using the column names 
pnt_df_tree_10 = pnt.merge(df_tree_10, left_on='id', right_on='key_0')
# transform Dataframe to Geodataframe
pnt_gdf_tree_10 = gpd.GeoDataFrame(pnt_df_tree_10,geometry='geometry')

In [59]:
# spatial join 2 Geodataframe : Basic cbg Geodataframe (based) and tree Geodataframe by using "how='left'"
sjoin_tree_10 = gpd.sjoin(left_df=cbg, right_df=pnt_gdf_tree_10,how='left')
df_tree_10 = sjoin_tree_10.groupby(['GEOID10'])['tree_count_10m'].mean()#.reset_index()
df_tree_10 #Series

C:\Users\skfh\Anaconda3\envs\GEOG573\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(epsg:4326 != None)
  "(%s != %s)" % (left_df.crs, right_df.crs)


GEOID10
530330001001    1.750000
530330001002    1.250000
530330001003    1.629630
530330001004    1.500000
530330001005    1.684211
                  ...   
530330121002    1.000000
530330260011    2.000000
530330264004         NaN
530330264005         NaN
530330265001    1.000000
Name: tree_count_10m, Length: 481, dtype: float64

# Step5 (Finally!): Append columns(Dataframe or Series) to the original cbg (Dataframe)

In [80]:
# method 1: using merge - add Dataframe to based Dataframe, using "on='key'""
cbg_final = cbg.merge(df_dis_ospace_m,on='GEOID10')
cbg_final = cbg_final.merge(df_speedlimit,on='GEOID10')


# method 2: using df['key'].map(Series) - add Series to df
cbg_final['crim_50m'] = cbg_final['GEOID10'].map(df_crim_50)
cbg_final['tree_10m'] = cbg_final['GEOID10'].map(df_tree_10)
cbg_final

,GEOID10,TRACTCE10,BLKGRPCE10,NAMELSAD10,INTPTLAT10,INTPTLON10,TRACT,TRBG,TRBG_STR,ACRES_TOTA,ACRES_LAND,ACRES_WATE,WATER,geometry,dis_ospace_m,SPEEDLIMIT,crim_50m,tree_10m
0,530330001001,000100,1,Block Group 1,+47.7276870,-122.2815160,100,100.1,100.1,588.283002,221.041200,367.285577,0,"POLYGON ((-122.26840 47.72641, -122.26719 47.7...",194.450279,20.176056,1.830189,1.750000
1,530330001002,000100,2,Block Group 2,+47.7293165,-122.2924690,100,100.2,100.2,71.258030,71.263215,0.000000,0,"POLYGON ((-122.28970 47.73193, -122.28965 47.7...",292.763196,23.868421,5.296296,1.250000
2,530330001003,000100,3,Block Group 3,+47.7228093,-122.2894558,100,100.3,100.3,91.604629,91.611355,0.000000,0,"POLYGON ((-122.28633 47.72093, -122.28629 47.7...",221.587445,23.880597,11.650000,1.629630
3,530330001004,000100,4,Block Group 4,+47.7319367,-122.2944531,100,100.4,100.4,31.237245,31.239309,0.000000,0,"POLYGON ((-122.29237 47.73194, -122.29231 47.7...",140.243242,25.107527,8.285714,1.500000
4,530330001005,000100,5,Block Group 5,+47.7229283,-122.2944374,100,100.5,100.5,56.956546,56.960756,0.000000,0,"POLYGON ((-122.29273 47.72019, -122.29273 47.7...",270.273622,24.865591,21.243902,1.684211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,530330117003,011700,3,Block Group 3,+47.5254231,-122.2847614,11700,11700.3,11700.3,125.007482,125.008883,0.000000,0,"POLYGON ((-122.28084 47.52520, -122.28082 47.5...",184.617027,21.820175,2.259740,1.616667
477,530330117004,011700,4,Block Group 4,+47.5148776,-122.2755994,11700,11700.4,11700.4,213.856873,213.859082,0.000000,0,"POLYGON ((-122.27932 47.52258, -122.27954 47.5...",213.080046,21.631098,3.547945,1.916667
478,530330118001,011800,1,Block Group 1,+47.5154414,-122.2532080,11800,11800.1,11800.1,648.593806,87.868697,560.733013,0,"POLYGON ((-122.26232 47.53173, -122.24933 47.5...",80.630733,20.801282,3.470588,2.000000
479,530330118002,011800,2,Block Group 2,+47.5159969,-122.2571162,11800,11800.2,11800.2,92.839709,92.840704,0.000000,0,"POLYGON ((-122.25464 47.51692, -122.25455 47.5...",210.383607,22.009524,2.181818,1.923077


In [81]:
cbg_final.shape

(481, 18)

In [83]:
#drop these 2 cbg polygon with no locaiton data within 
cbg_final_dropped = cbg_final[cbg_final.GEOID10 != '530330264004']
cbg_final_dropped = cbg_final_dropped[cbg_final_dropped.GEOID10 != '530330264005']
cbg_final_dropped.shape

(479, 18)

# Export the Geodaraframe to either .csv or .shp

In [85]:
cbg_final_dropped.to_csv('seattle_urban_features.csv') 
cbg_final_dropped.to_file('seattle_urban_features.shp')
#so 479 cbg in Seattle city in this research. Hurrrrrrrrray!